In [ ]:
import sympy as sp
import numpy as np
from sympy import *
init_printing()

# Introduction

This script uses SymPy to calculate the expansion function $\Theta$ symbolically. It requires the 3+1 ADM components $\alpha, \beta$ and $\psi^4$ as input. Note that it assumes a spherical coordinate system.

# Initializing

In [ ]:
# Initialize coordinates (spherical)
r, theta, phi = symbols('r, theta, phi')
coords = [r, theta, phi]

########################################################################################################################################################

#####################
## Choose symmetry ##
#####################
    # var = 0 --> spherical symmetry
    # var = 1 --> phi symmetry (theta case)
    # var = 2 --> theta symmetry (phi case)
    # var = 3 --> no symmetry (general case)

var = 3

if var == 0:
    h = Function('h')
    s = Array([1, 0, 0])
elif var == 1:
    h = Function('h')(theta)
    s = Array([1, -diff(h, theta), 0])
elif var == 2:
    h = Function('h')(phi)
    s = Array([1, 0, -diff(h, phi)])
elif var == 3:
    h = Function('h')(theta, phi)
    s = Array([1, -diff(h, theta), -diff(h, phi)])
    

########################################################################################################################################################

# Additional symbols
N = symbols('N')
M = symbols('M')
R = symbols('R')


########################################################################################################################################################


######################################
## ADM 3+1 components of the metric ##
######################################

alpha = Function('alpha')(r, theta, phi)   # lapse function
alpha = (1-M/(2*r)) / (1+M/(2*r))

beta = Array([0,0,0])                       # Shift vector

psi = Function('psi')(r, theta, phi)       # Conformal factor
psi = 1 + M/(2*r)
psi4 = psi**4


eta = diag(1, r**2, (r*sin(theta))**2)
eta_inv = eta**(-1)

# Normal vectors

Normal vectors defined by $$n^{\mu} = \frac{1}{\alpha} (1, -\beta^i)$$ such that
$$n^i = -\frac{1}{\alpha} \beta^i$$ and $$n_i = \gamma_{ij}n^j = \psi^4\eta_{ij}n^j = -\psi^4\frac{1}{\alpha}\eta_{ij}\beta^j$$

In [ ]:
'''
Define normal vectors
'''

# Index up
def norm_up(alpha, beta, coords):
    dim = len(coords)
    n_up = sp.MutableDenseNDimArray.zeros(dim)
    for i in range(dim):
        n_up[[i]] = - 1/alpha * beta[i]
    return n_up

# Index down
def norm_down(alpha, beta, psi4, eta, coords):
    dim = len(coords)
    n_down = sp.MutableDenseNDimArray.zeros(dim)
    for i in range(dim):
        n_down[[i]] = - psi4 * 1/alpha * sum(eta[i, j] * beta[j] for j in range(dim))
    return n_down

# Christoffel symbols

In the conformally flat assumption $$\gamma^{ij} = \psi^{-4} \eta^{ij}$$ the Christoffel symbols become
$$\begin{aligned}
\Gamma^i_{jk} &= \frac{1}{2} \gamma^{ia} (\partial_j \gamma_{ka} + \partial_k \gamma_{ja} - \partial_a \gamma_{jk}) \\[5pt]
&= \frac{1}{2} \psi^{-4} \eta^{ia} \left[\partial_j(\psi^4\eta_{nk}) + \partial_k(\psi^4\eta_{ja}) - \partial_a(\psi^4\eta_{jk})\right] \\[5pt]
&= \frac{1}{2} \eta^{ia} (\partial_j \eta_{ka} + \partial_k \eta_{ja} - \partial_a \eta_{jk}) + \frac{1}{2} \psi^{-4} \eta^{ia} (\eta_{ka} \partial_j \psi^4 + \eta_{ja} \partial_k \psi^4 - \eta_{jk} \partial_a \psi^4)
\end{aligned}$$

In [ ]:
'''
Christoffel symbols
'''

def christoffel_symbols(psi4, eta, eta_inv, coords):
    n = len(coords)
    Gamma = sp.MutableDenseNDimArray.zeros(n, n, n)
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                Gamma[i, j, k] = (0.5 * sum([eta_inv[i, a] * (sp.diff(eta[k, a], coords[j]) + 
                                                           sp.diff(eta[j, a], coords[k]) - 
                                                           sp.diff(eta[j, k], coords[a])) for a in range(n)]) +
                                 0.5 * 1/psi4 * sum([eta_inv[i, a] * (eta[k, a] * sp.diff(psi4, coords[j]) + 
                                                                       eta[j, a] * sp.diff(psi4, coords[k]) - 
                                                                       eta[j, k] * sp.diff(psi4, coords[a])) for a in range(n)]))
    return Gamma

# Vector s

Vector $s$ given by $$s_i = \lambda (1, -\partial_{\theta}h, \partial_{\phi}h)$$ such that $$s^i = \gamma^{ij} s_j = \psi^{-4} \eta^{ij} s_j \quad \Leftrightarrow \quad s^i = \lambda \psi^{-4} \left(1, -\frac{1}{r^2} \partial_{\theta}h, -\frac{1}{r^2 \sin^2{\theta}} \partial_{\phi}h\right)$$ and $$1 = s_i s^i = \lambda^2 \psi^{-4} \left(1 + \frac{1}{r^2} (\partial_{\theta}h)^2 + \frac{1}{r^2 \sin^2{\theta}} (\partial_{\phi}h)^2\right)$$

In [ ]:
'''
Spacelike outward-pointing vector s
'''

def s_norm(h, s, psi4, coords, eta_inv):
    n = len(coords)
    s_inv = sp.MutableDenseNDimArray.zeros(n)
    for i in range(n):
        s_inv[[i]] = (1/psi4) * sum([eta_inv[i, j] * s[j] for j in range(n)])
    norm = 1 / (sum(s[i] * s_inv[i] for i in range(n)))
    N = sp.sqrt(norm)
    s_norm = N * s
    s_inv_norm = N * s_inv
    return s_inv_norm

# Extrinsic curvature

Calculating from the formula $$K_{ij} = \gamma_i^a \gamma_j^b \nabla_a n_b$$ where $$\gamma_i^a = \delta_i^a + n^a n_i$$

In [ ]:
'''
Extrinsic Curvature
'''

def extrinsic_curvature(n_up, n_down, christoffel, coords):
    n = len(coords)
    delta = eye(n)
    K = sp.MutableDenseNDimArray.zeros(n, n)
    for i in range(n):
        for j in range(n):
            K[i, j] = sum(sum((delta[i, a] + n_up[a]*n_down[i]) * (delta[j, b] + n_up[b]*n_down[j]) * (diff(n_down[b], coords[a]) -
                      sum(christoffel[k, a, b]*n_down[k] for k in range(n))) for b in range(n)) for a in range(n))
    return K

# Expansion equation

The expansion equation is $$\Theta = \nabla_i s^i + s^i s^j K_{ij} - K = 0$$
We assume maximal slicing condition such that $$K = 0$$

In [ ]:
'''
Expansion function
'''

# First term
def cov_derivative(christoffel, s, coords):
    n = len(coords)
    cov_dev = sum(diff(s[i], coords[i]) for i in range(n)) + sum(sum(christoffel[i, i, j] * s[j] for j in range(n)) for i in range(n))
    return cov_dev

# Second term
def curv(ext_curv, s, coords):
    n = len(coords)
    curv = sum(sum(ext_curv[i, j] * s[i] * s[j] for j in range(n)) for i in range(n))
    return curv

# Full equation
def expansion(christoffel, s, ext_curv, coords):
    return (cov_derivative(christoffel, s, coords) + curv(ext_curv, s, coords))

In [ ]:
#test = expansion(christoffel, s_inv_norm, cur, coords)
#display(sp.simplify(test))

***

***

***

***